Upgrade RAM in Colab

In [0]:
# upgrade ram
a = []
while(1):
    a.append('1'*100000)

In [0]:
%tensorflow_version 1.x

# Initialize Tensorflow Object Detection API interface

Refs: 
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [0]:
%cd /content
!git clone --quiet https://github.com/PeppeDAlterio/android-object-detection.git /content/api

!mv /content/api/generate_tfrecord.py /content/generate_tfrecord.py
!mv /content/api/xml_to_csv.py /content/xml_to_csv.py
!mv /content/api/data_augmentation.py /content/data_augmentation.py
!mv /content/api/train_val_split.py /content/train_val_split.py

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib pycocotools tf_slim numpy==1.16

%cd /content/api/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/api/research/:/content/api/research/slim/'

!python object_detection/builders/model_builder_test.py

%cd /content

In [0]:
# Model to use (see MODELS_CONFIG)
selected_model = 'ssd_mobilenet_v2_coco'

# List of models (from Tensorflow Zoo-Garden)
MODELS_CONFIG = {
    'ssd_mobilenet_v2_coco': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29', #http://download.tensorflow.org/models/object_detection/{NOME_MODELLO}.tar.gz
        'pipeline_file': 'pipeline.config', # Dall'archivio ^
        'batch_size': 32
    }
}

MODEL = MODELS_CONFIG[selected_model]['model_name']
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']
batch_size = MODELS_CONFIG[selected_model]['batch_size']

# Prepare the dataset

In [0]:
!mkdir /content/data
!mkdir /content/data/images
!mkdir /content/data/images/train
!mkdir /content/data/images/val
!mkdir /content/data/images/test
!mkdir /content/data/annotations
!mkdir /content/data/images/all



1.   Collect the images
2.   Augment the data by running *data_augmentation.py* script
3.   Label the dataset using a labelling tool such as labelImg (https://github.com/tzutalin/labelImg). Please be sure to use a proper XML format (e.g. Pascal VOC)
4.   Move images and labels (xml) into */content/data/images/all/*


## Split your dataset in training e validation set.
You can also do this automatically by running the first code block

In [0]:
!python train_val_split.py \
        --datadir='/content/data/images/all' \
        --split=0.1 \
        --train_output='/content/data/images/train' \
        --test_output='/content/data/images/val' \
        --image_ext='jpg'

Compute number of images in training and validation set

In [0]:
import os
import math

num_train_images = 0
num_val_images = 0

for image in os.listdir('/content/data/images/train'):
  if image.endswith('xml'):
    num_train_images = num_train_images + 1

for image in os.listdir('/content/data/images/val'):
  if image.endswith('xml'):
    num_val_images = num_val_images + 1

print ("Number of images in train set: " + str(num_train_images) )
print ("Number of images in val set: " + str(num_val_images) )

steps_per_epoch = math.ceil(num_train_images/batch_size)
print("Steps per epoch: " + str(steps_per_epoch) )

Generate TFRecord from both train and val set

In [0]:
%cd /content/
# Path file da generare
val_record_fname = '/content/data/annotations/val.record'
train_record_fname = '/content/data/annotations/train.record'
label_map_pbtxt_fname = '/content/data/annotations/label_map.pbtxt'

# Conversione coppie (jpg,xml) in un unico csv e generazione mappa dei label
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv \
                      --labelMapDir /content/data/annotations/
!python xml_to_csv.py -i data/images/val -o data/annotations/val_labels.csv

# Training set
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Validation set
!python generate_tfrecord.py --csv_input=data/annotations/val_labels.csv --output_path=data/annotations/val.record --img_path=data/images/val --label_map data/annotations/label_map.pbtxt

# Download and init model

Ref: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [0]:
# IoU threshold from the original configuration of non-max suppression
IOU_OLD_THRESHOLD = 0.6

# New IoU threshold for non-max suppression
IOU_THRESHOLD = 0.55

# Number of steps
num_steps = 10000 # steps_per_epoch*epochs # 200000

# Number of evaluation steps. Comment to use the whole dataset
# num_eval_steps = 50

num_evals_samples = 100000

%cd /content/api/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/api/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)


!echo {DEST_DIR}
!ls -alh {DEST_DIR}

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

import os

!mkdir /content/tmp
!cp {DEST_DIR}/{pipeline_file} /content/tmp/config.cfg
pipeline_fname = '/content/tmp/config.cfg'

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set number of images in val set
    s = re.sub('max_evals: [0-9]+',
               'max_evals: {}'.format(num_evals_samples), s)
    
    ### Batch non-max suppression ###

    # Set score threshold
    s = re.sub('score_threshold: ".*?"',
               'score_threshold: {}'.format("1e-8"), s)
    
    # Set IoU threshold
    s = re.sub('iou_threshold: ' + str(IOU_OLD_THRESHOLD),
               'iou_threshold: {}'.format(str(IOU_THRESHOLD)), s)
    
    # Rimozione batch_norm_trainable
    s = re.sub('batch_norm_trainable: true', '', s)
    s = re.sub('batch_norm_trainable: false', '', s)

    f.write(s)

!cat {pipeline_fname}

# Estimating optimal learning rate
If you do not know what's this about, skip this section

In [0]:
lr_from = -2.39794000662
lr_to = -2.16
print("From: " + str(10**lr_from) )
print("To: " + str(10**lr_to) )

In [0]:
import random
import re

# Output model directory
model_dir = '/content/fit_output'

MAX_RANGE = 10

for i in range(0,MAX_RANGE):
  # Intervallo ricerca learning rate
  lr = 10**random.uniform(lr_from, lr_to)
  # lr = ... # FINER 

  with open(pipeline_fname) as f:
    s = f.read()
  with open(pipeline_fname, 'w') as f:
          
    # Set learning rate
    s = re.sub('initial_learning_rate: [0-9.]+',
              'initial_learning_rate: {}'.format(str(lr)), s)

    f.write(s)

  #!cat {pipeline_fname}

  print('********** lr = '+str(lr)+' **********')

  !python /content/api/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir={model_dir} \
      --num_train_steps=120 \
      --save_checkpoints_steps=200 \
      --log_step_count_steps=20 \
      --keep_checkpoint_max=0 \
      --eval_throttle_secs=30 \
      --eval_start_delay_secs=30

  !rm -r {model_dir}

In [0]:
# Learning rate
lr = 0.006397486877114805 #0.00400000018999

with open(pipeline_fname) as f:
  s = f.read()
with open(pipeline_fname, 'w') as f:
        
  # Set learning rate
  s = re.sub('initial_learning_rate: [0-9.]+',
             'initial_learning_rate: {}'.format(str(lr)), s)

  f.write(s)

!cat {pipeline_fname}

# Fit

In [0]:
# Output model directory
model_dir = '/content/fit_output'

# !!! Clean previous fit !!!
!rm -r {model_dir}

!mkdir {model_dir}

# 10'000 -> 500
save_checkpoints_steps = int(num_steps/40)

# 10'000 -> 500
log_step_count_steps = int(num_steps/40)

# Maximum number of checkpoints to keep in storage. The oldest one are removed
# newest are kept. 0 or None disables this feature.
keep_checkpoint_max = 5

# Evaluate every N seconds
eval_throttle_secs = 480

# Start evaluating after N seconds
eval_start_delay_secs = 120

!python /content/api/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --num_train_steps={num_steps} \
    --save_checkpoints_steps={save_checkpoints_steps} \
    --log_step_count_steps={log_step_count_steps} \
    --keep_checkpoint_max={keep_checkpoint_max} \
    --eval_throttle_secs={eval_throttle_secs} \
    --eval_start_delay_secs={eval_start_delay_secs}

# Deploy

## Export TFLite

In [0]:
import re
import numpy as np

!mkdir /content/tflite_output
tflite_output_directory = '/content/tflite_output'

#lst = os.listdir(model_dir)
#lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
#steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
#last_model = lst[steps.argmax()].replace('.meta', '')

last_model = 'model.ckpt-4500'

last_model_path = os.path.join(model_dir, last_model)

print(last_model_path)

!python /content/api/research/object_detection/export_tflite_ssd_graph.py \
                --pipeline_config_path={pipeline_fname} \
                --trained_checkpoint_prefix={last_model_path} \
                --output_directory={tflite_output_directory} \
                --add_postprocessing_op=true

!tflite_convert --graph_def_file={tflite_output_directory}/tflite_graph.pb \
                --output_file={tflite_output_directory}/detect.tflite \
                --input_arrays=normalized_input_image_tensor \
                --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
                --input_shapes=1,300,300,3 \
                --inference_type=FLOAT \
                --mean_values=128 \
                --std_dev_values=127 \
                --change_concat_input_ranges=false \
                --allow_custom_ops